In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt

from src.dataset import RUL_Dataset
from src.model import CNN1D_RUL
from src.RUL_loss_function import RUL_loss

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

In [2]:
training_dataset = RUL_Dataset(train_dir="D:\Imperial_College\FYP\B0_tests\Training", permutations=10 )
testing_dataset = RUL_Dataset(train_dir="D:\Imperial_College\FYP\B0_tests\Testing", permutations=2 )

training_loader = DataLoader(training_dataset, batch_size=1, shuffle=False)
testing_loader = DataLoader(testing_dataset, batch_size=1, shuffle=False)

Sign : ' ' does not work
Sign : ',' works
Sign : ';' does not work
Sign : '    ' does not work
Sign : ' ' does not work
Sign : ',' works
Sign : ';' does not work
Sign : '    ' does not work
Sign : ' ' does not work
Sign : ',' works
Sign : ';' does not work
Sign : '    ' does not work
Sign : ' ' does not work
Sign : ',' works
Sign : ';' does not work
Sign : '    ' does not work
Sign : ' ' does not work
Sign : ',' works
Sign : ';' does not work
Sign : '    ' does not work


In [26]:
RUL_model = CNN1D_RUL()
RUL_model = RUL_model.to(device)

optimizer = torch.optim.Adam(RUL_model.parameters(), lr=0.001)


In [22]:
test_forw = next(iter(training_loader))

In [25]:
features, labels = test_forw
labels = labels[0].to(device)
features = features[0].to(device)

outputs = RUL_model(features)

outputs

tensor([[0.4844],
        [0.4858]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [27]:
loss = loss_func(outputs)

loss

TypeError: RUL_loss.forward() missing 2 required positional arguments: 'true' and 'theta'

In [ ]:
# On3 training loop

def train_1_epoch(model, training_loader, loss_func, ):
    
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of iter(training_loader) so that we can track the batch index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        labels = labels[0].to(device)
        inputs = inputs[0].to(device)

        # Zero gradients for every batch
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss